In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re

In [2]:
from sqlalchemy import create_engine

username = 'admin'
password = 'nX0d4tuvxL3w5hMjJRfz'
host = 'ai-engine.cluster-cla9jbugaywl.eu-west-2.rds.amazonaws.com'
database_name = 'superleague_clubs'

connection_url = f'mysql+pymysql://{username}:{password}@{host}/{database_name}'

def insert_into_db(df, table_name) : 

    engine = create_engine(connection_url)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False) 
    engine.dispose()
    return 'Data inserted successfully'

In [8]:
def remove_square_brackets(text):
    return re.sub(r'\[.*?\]', '', text)

In [42]:
url = 'https://www.soccerbase.com/teams/team.sd?team_id=1370&teamTabs=managers'

In [43]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu") 

driver = webdriver.Chrome(options=chrome_options)  
driver.get(url)
driver.implicitly_wait(5)

page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")

driver.quit()

In [44]:
df = pd.read_html(page_source)
managers = df[1]

In [38]:
drop_cols = ['M', 'W', 'D', 'L', 'GF', 'GA', 'Win%[1]', 'Honours', 'Notes']
managers = managers.drop(columns = drop_cols)

In [40]:
managers['Manager'] = managers['Manager'].apply(remove_square_brackets)

In [46]:
insert_into_db(managers, 'intermilan_managers')

'Data inserted successfully'

In [45]:
managers.head()

,Unnamed: 0,FROM,TO
0,Simone Inzaghi,"07 Jul, 2021",Present
1,Antonio Conte,"31 May, 2019","07 Jul, 2021"
2,Luciano Spalletti,"09 Jun, 2017","01 May, 2019"
3,Stefano Vecchi,"09 May, 2017","09 Jun, 2017"
4,Stefano Pioli,"08 Nov, 2016","08 May, 2017"
